In [1]:
%load_ext autoreload
%autoreload 2
import os
os.chdir("../")

In [2]:
import torch
import torch.nn.functional as F
from xent.tasks import Closure
from xent.models import M
from xent.lang import X
from xent.dataprocessing import Wikipedia
from xent.config import *

In [3]:
model = M("gpt2", "M0", base="base")
checker_model = M("gpt2", "M1-zero", base="closure")

In [4]:
corpus_generator = Wikipedia(split=0.8)
get_test_sample = corpus_generator.get_random_test_text

Loading dataset shards:   0%|          | 0/41 [00:00<?, ?it/s]

In [5]:
task = Closure(model)
synth = task.generate(get_test_sample, space="tokens")

In [6]:
cut = task.find_xstring(synth, X.xreturn)
CUT = cut + 6

In [7]:
print(model.detokenize(synth[0, CUT+2::4]))
print(model.detokenize(synth[0, CUT:]))

 9 7 4 4 2 4 2 8 0 3 14 2 2 11 4 3 4 5 2 5 1 1 0 0 2 0 7 1 10 7 29 7 2 5 0 2 0 5 1 0 4 1 1 25 1 0 0 7 1 5 1 2 24 8 3 0 0 0 6 2 4 0 0 0 4 0 1 22 4 0 0 0 0 2 0 1 0 0 0 0 0 1 16 1 0 9 3 5 0 9 5 5 1 0 8 0 0 1 1 1 0
oh: 9
wa: 7
-: 4
d: 4
ong: 2
 is: 4
 a: 2
 d: 8
ong: 0
,: 3
 neighbourhood: 14
 of: 2
 the: 2
 Map: 11
o: 4
-: 3
gu: 4
 district: 5
 in: 2
 Seoul: 5
,: 1
 South: 1
 Korea: 0
.: 0

: 2

: 0
See: 7
 also: 1
 : 10

: 7
 Administrative: 29
 divisions: 7
 of: 2
 South: 5
 Korea: 0

: 2

: 0
References: 5

: 1

: 0
External: 4
 links: 1

: 1
 Map: 25
o: 1
-: 0
gu: 0
 official: 7
 website: 1
 in: 5
 English: 1

: 2
 Map: 24
 of: 8
 Map: 3
o: 0
-: 0
gu: 0
 at: 6
 the: 2
 Map: 4
o: 0
-: 0
gu: 0
 official: 4
 website: 0

: 1
 Map: 22
 of: 4
 Map: 0
o: 0
-: 0
gu: 0
 at: 2
 the: 0
 Map: 1
o: 0
-: 0
gu: 0
 official: 0
 website: 0

: 1
 D: 16
oh: 1
wa: 0
 1: 9
-: 3
d: 5
ong: 0
 resident: 9
 office: 5
 website: 5

: 1

: 0
Neigh: 8
bour: 0
hood: 0
s: 1
 of: 1
 Map: 1
o: 0



In [8]:
checker_model.model.eval()
with torch.no_grad():
    print(synth.shape)
    logits = checker_model.model(synth).logits
    loss = F.cross_entropy(logits[0, CUT:-1], synth[0, CUT+1:], reduction="none")
    # Get the predicted token probabilities
    probs = F.softmax(logits[0, CUT:-1], dim=-1)
    # Get the indices of tokens with highest probabilities
    highest_prob_tokens = torch.argmax(probs, dim=-1)
    # Convert to list for easier inspection

torch.Size([1, 522])


In [18]:
word_origin = synth[0, CUT+4::4]
origin = synth[0, CUT+4+2::4]
word_genera = highest_prob_tokens[3::4]
genera = highest_prob_tokens[3+2::4]

print(f"{'actual':20}  | {'generated':20}")
print("----------------------|---------------------")
for wo, o, wg, g in zip(word_origin, origin, word_genera, genera):
    wo_str = model.detokenize(wo)
    o_str = model.detokenize(o)
    wg_str = model.detokenize(wg) 
    g_str = model.detokenize(g)
    if '\n' in wo_str: wo_str = "\\n"
    if '\n' in wg_str: wg_str = "\\n"
    print(f"{wo_str:15} {o_str:5} | {wg_str:15} {g_str:8}")

actual                | generated           
----------------------|---------------------
wa               7    | wa               6      
-                4    | -                4      
d                4    | d                5      
ong              2    | ong              1      
 is              4    |  is              4      
 a               2    |  a               2      
 d               8    |  d               8      
ong              0    | ong              4      
,                3    | ,                3      
 neighbourhood   14   |  neighbourhood   11     
 of              2    |  of              1      
 the             2    |  the             2      
 Map             11   |  Map             10     
o                4    | o                6      
-                3    | -                3      
gu               4    | gu               6      
 district        5    |  district        3      
 in              2    |  in              2      
 Seoul           5    |  Seo